In [1]:
using Pkg; Pkg.activate("."); Pkg.instantiate();

using Flux, Flux.Data.MNIST
using Flux: @epochs, onehotbatch, mse, throttle
using Base.Iterators: partition
using Juno: @progress
using CuArrays

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


Encode MNIST images as compressed vectors that can later be decoded back into
images.

In [2]:
imgs = MNIST.images()

60000-element Array{Array{ColorTypes.Gray{FixedPointNumbers.Normed{UInt8,8}},2},1}:
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8

Partition into batches of size 1000

In [3]:
data = [float(hcat(vec.(imgs)...)) for imgs in partition(imgs, 1000)]
data = gpu.(data)

N = 32 # Size of the encoding

32

You can try to make the encoder/decoder network larger
Also, the output of encoder is a coding of the given input.
In this case, the input dimension is 28^2 and the output dimension of
encoder is 32. This implies that the coding is a compressed representation.
We can make lossy compression via this `encoder`.

In [4]:
encoder = Dense(28^2, N, leakyrelu) |> gpu
decoder = Dense(N, 28^2, leakyrelu) |> gpu

m = Chain(encoder, decoder)

loss(x) = mse(m(x), x)

evalcb = throttle(() -> @show(loss(data[1])), 5)
opt = ADAM(params(m))

@epochs 10 Flux.train!(loss, zip(data), opt, cb = evalcb)

┌ Info: Epoch 1
└ @ Main.##657 /home/dhairyagandhi96/.julia/packages/Flux/kJVo6/src/optimise/train.jl:62
loss(data[1]) = 0.103790335f0 (tracked)
┌ Info: Epoch 2
└ @ Main.##657 /home/dhairyagandhi96/.julia/packages/Flux/kJVo6/src/optimise/train.jl:62
┌ Info: Epoch 3
└ @ Main.##657 /home/dhairyagandhi96/.julia/packages/Flux/kJVo6/src/optimise/train.jl:62
┌ Info: Epoch 4
└ @ Main.##657 /home/dhairyagandhi96/.julia/packages/Flux/kJVo6/src/optimise/train.jl:62
┌ Info: Epoch 5
└ @ Main.##657 /home/dhairyagandhi96/.julia/packages/Flux/kJVo6/src/optimise/train.jl:62
┌ Info: Epoch 6
└ @ Main.##657 /home/dhairyagandhi96/.julia/packages/Flux/kJVo6/src/optimise/train.jl:62
┌ Info: Epoch 7
└ @ Main.##657 /home/dhairyagandhi96/.julia/packages/Flux/kJVo6/src/optimise/train.jl:62
┌ Info: Epoch 8
└ @ Main.##657 /home/dhairyagandhi96/.julia/packages/Flux/kJVo6/src/optimise/train.jl:62
┌ Info: Epoch 9
└ @ Main.##657 /home/dhairyagandhi96/.julia/packages/Flux/kJVo6/src/optimise/train.jl:62
┌ Info: Epoch 1

Sample output

In [5]:
using Images

img(x::Vector) = Gray.(reshape(clamp.(x, 0, 1), 28, 28))

function sample()
  # 20 random digits
  before = [imgs[i] for i in rand(1:length(imgs), 20)]
  # Before and after images
  after = img.(map(x -> cpu(m)(float(vec(x))).data, before))
  # Stack them all together
  hcat(vcat.(before, after)...)
end

cd(@__DIR__)

#save("sample.png", sample())